## Floating errors

Documented here are some bugs found while compiling and running CLIRAD-SW using different combinations of the compiler's debug flags.

The compiler used here is:

In [19]:
!ifort -v

Version 11.1 


These are the files containing the code and the data:

In [22]:
!ls

cah.data.f	     coa.data.f     mls75.pro		solar_code.f
cldreff.f	     fort.9.txt     mls75_pro.f
cldreff.f~	     mainsw_2008.f  ReadMe_SW code.txt
cliradsw_bugs.ipynb  mcai.data.f    sfcref.f


A simple compilation gives no errors and clirad returns legible numbers:

In [23]:
!ifort sfcref.f cldreff.f solar_code.f mainsw_2008.f -o clirad.exe

In [24]:
!./clirad.exe

In [25]:
!tail -n 20 fort.9

                                          1.502
  71  894.1250  201.5172   406.7708
                                          0.589
  72  917.9000  199.8582   402.8219
                                          0.593
  73  941.6750  198.1888   398.8942
                                          0.592
  74  965.4500  196.5212   395.0023
                                          0.588
  75  989.2250  194.8641   391.1545
                                          0.597
  76 1013.0000  193.1818   387.2902


 fdiruv  =    2.49
 fdifuv  =   13.59
 fdirpar =   40.37
 fdifpar =   79.88
 fdirir  =   44.93
 fdifir  =   61.90


However, if debugging flags are used to check for floating point errors, an error is raised and is traced back to line 335 in solar_code.f:

In [26]:
!ifort -O0 -g -traceback -check all -fpe0 sfcref.f cldreff.f solar_code.f mainsw_2008.f -o clirad.exe

In [27]:
!./clirad.exe

forrtl: error (73): floating divide by zero
Image              PC                Routine            Line        Source             
clirad.exe         00000000005314BA  Unknown               Unknown  Unknown
clirad.exe         000000000041A1CB  sorad_                    335  solar_code.f
clirad.exe         00000000004BBD47  MAIN__                    319  mainsw_2008.f
clirad.exe         000000000040322C  Unknown               Unknown  Unknown
libc.so.6          0000003705A1D994  Unknown               Unknown  Unknown
clirad.exe         0000000000403139  Unknown               Unknown  Unknown


That line is:

In [15]:
!sed -n '335, 335p' solar_code.f

          swh(i,k)=log10(pl(i,k))


### Forcing all reals to be of double precision at time of compilation

By using the `-r8` flag at compilation, all real variables are forced to have double precision, regardless of their declared precision in the code.  Here shows what happens when this is used.

If we do not check for floating point errors, clirad compiles fine but returns NaN values:

In [28]:
!ifort -O0 -g -traceback -check all -r8 sfcref.f cldreff.f solar_code.f mainsw_2008.f -o clirad.exe

In [29]:
!./clirad.exe

In [30]:
!tail -n 20 fort.9

                                            NaN
  71  894.1250       NaN        NaN
                                            NaN
  72  917.9000       NaN        NaN
                                            NaN
  73  941.6750       NaN        NaN
                                            NaN
  74  965.4500       NaN        NaN
                                            NaN
  75  989.2250       NaN        NaN
                                            NaN
  76 1013.0000       NaN        NaN


 fdiruv  =    0.00
 fdifuv  =     NaN
 fdirpar =    0.00
 fdifpar =     NaN
 fdirir  =     NaN
 fdifir  =     NaN


If we do check for floating point errors, then an error is raised and traced to line 1626 in solar_code.f:

In [31]:
!ifort -O0 -g -traceback -check all -fpe0 -r8 sfcref.f cldreff.f solar_code.f mainsw_2008.f -o clirad.exe

In [32]:
!./clirad.exe

forrtl: error (65): floating invalid
Image              PC                Routine            Line        Source             
clirad.exe         000000000046D285  deledd_                  1626  solar_code.f
clirad.exe         0000000000433491  soluv_                    726  solar_code.f
clirad.exe         0000000000415290  sorad_                    231  solar_code.f
clirad.exe         00000000004BBD5F  MAIN__                    319  mainsw_2008.f
clirad.exe         000000000040314C  Unknown               Unknown  Unknown
libc.so.6          0000003705A1D994  Unknown               Unknown  Unknown
clirad.exe         0000000000403059  Unknown               Unknown  Unknown


That line is:

In [16]:
!sed -n '1626, 1626p' solar_code.f

	tau(i,k) = dble(tau1(i,k))
